<a href="https://colab.research.google.com/github/Yashsharma009/new/blob/main/geolocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install exif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
from exif import Image
img_path = '/content/drive/MyDrive/dataset-plastic1/train/images/DJI_0016.jpg'
with open(img_path, 'rb') as src:
    img = Image(src)
    print (src.name, img)

/content/drive/MyDrive/dataset-plastic1/train/images/DJI_0016.jpg <exif._image.Image object at 0x7da78cec37c0>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import PIL
image = PIL.Image.open(img_path)
image

In [ ]:
if img.has_exif:
 info = f” has the EXIF {img.exif_version}”
else:
 info = “does not contain any EXIF information”
print(f”Image {src.name}: {info}”)

SyntaxError: ignored

In [6]:
from PIL import Image
from PIL.ExifTags import TAGS

# path to the image or video
imagename = "/content/drive/MyDrive/dataset-plastic1/train/images/DJI_0016.jpg"

# read the image data using PIL
image = Image.open(imagename)

# extract EXIF data
exifdata = image.getexif()

# iterating over all EXIF data fields
for tag_id in exifdata:
    # get the tag name, instead of human unreadable tag id
    tag = TAGS.get(tag_id, tag_id)
    data = exifdata.get(tag_id)
    # decode bytes
    if isinstance(data, bytes):
        data = data.decode()
    print(f"{tag:25}: {data}")

GPSInfo                  : 804
ResolutionUnit           : 2
ExifOffset               : 212
Make                     : DJI
Model                    : FC300X
Software                 : Adobe Photoshop CC 2015 (Windows)
Orientation              : 1
DateTime                 : 2021:03:13 13:35:31
XResolution              : 240.0
YResolution              : 240.0


In [9]:
pip install gpsphoto

  Preparing metadata (setup.py) ... done
  Created wheel for gpsphoto: filename=gpsphoto-2.2.3-py3-none-any.whl size=11861 sha256=6944b0a12da0660dbd43e6df83119dcae7839f2173a91a2f82c13448b5aca2c8
  Stored in directory: /root/.cache/pip/wheels/f2/eb/59/24c9d0b0a97d5732b81f6dc509e4fbd7c74113aabf71079112
Successfully built gpsphoto


In [10]:
pip install GPSPhoto
import os

os.chdir('C:\\Users\\my_personal_profile\\image_folder')
image_list = os.listdir()
image_list = [a for a in image_list if a.endswith('jpg')]

print(image_list)

NotADirectoryError: ignored

In [11]:
pip install exif folium pandas numpy matplotlib

In [12]:
import json
from pathlib import Path

import folium
import matplotlib
import numpy as np
import pandas as pd
from exif import Image

In [13]:
def read_exif_data(file_path: Path) -> Image:
    """Read metadata from photo."""
    with open(file_path, 'rb') as f:
        return Image(f)

In [14]:
BASE_LOC = Path("/media/pav/Storage/Photo/Phone")
file = Path("/content/drive/MyDrive/dataset-plastic1/train/images/DJI_0016.jpg")

img = read_exif_data(base_loc / file)
print('\n'.join([i for i in img.list_all() if i.startswith('gps_')]))

NameError: ignored

In [15]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif(filename):
    exif = Image.open(filename)._getexif()

    if exif is not None:
        for key, value in exif.items():
            name = TAGS.get(key, key)
            exif[name] = exif.pop(key)

        if 'GPSInfo' in exif:
            for key in exif['GPSInfo'].keys():
                name = GPSTAGS.get(key,key)
                exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)

    return exif

exif = get_exif('/content/drive/MyDrive/dataset-plastic1/train/images/DJI_0016.jpg')

RuntimeError: ignored

In [16]:
!pip install Pillow


In [25]:
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_geotagging(exif):
    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

def convert_to_degrees(value):
    d = value[0]
    m = value[1]
    s = value[2]
    return d + (m / 60.0) + (s / 3600.0)

def get_gps_coordinates(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()

        if exif_data is not None:
            geotags = get_geotagging(exif_data)
            if geotags:
                latitude = geotags.get('GPSLatitude')
                longitude = geotags.get('GPSLongitude')

                if latitude and longitude:
                    latitude = convert_to_degrees(latitude)
                    longitude = convert_to_degrees(longitude)

                    if geotags.get('GPSLatitudeRef') == 'S':
                        latitude = -latitude
                    if geotags.get('GPSLongitudeRef') == 'W':
                        longitude = -longitude

                    return latitude, longitude

    except Exception as e:
        raise ValueError(f"Error while extracting GPS information from {image_path}") from e

    return None

# Replace 'your_folder_path' with the actual path to the folder containing images.
folder_path = '/content/drive/MyDrive/face/dataset-plastic1/train/images'

# Loop through each file in the folder and its subdirectories
for root, _, files in os.walk(folder_path):
    for filename in files:
        image_path = os.path.join(root, filename)
        coordinates = get_gps_coordinates(image_path)

        if coordinates:
            latitude, longitude = coordinates
            print(f"Image: {filename}, GPS Latitude: {latitude}, GPS Longitude: {longitude}")
        else:
            print(f"Image: {filename}, GPS information not found or invalid GPS coordinate format.")


Image: DJI_0017.jpg, GPS Latitude: 10.895858333333333, GPS Longitude: 106.69386833333333
Image: DJI_0016.jpg, GPS Latitude: 10.895855, GPS Longitude: 106.69395833333333
Image: DJI_0021.jpg, GPS Latitude: 10.895856666666667, GPS Longitude: 106.69388000000001
Image: DJI_0020.jpg, GPS Latitude: 10.895866666666667, GPS Longitude: 106.69366833333333
Image: DJI_0044.jpg, GPS Latitude: 10.895906666666667, GPS Longitude: 106.69513333333333
Image: DJI_0025.jpg, GPS Latitude: 10.895983333333334, GPS Longitude: 106.69356833333333
Image: DJI_0043.jpg, GPS Latitude: 10.895911666666667, GPS Longitude: 106.69506166666666
Image: DJI_0068.jpg, GPS Latitude: 10.895858333333333, GPS Longitude: 106.69387833333333
Image: DJI_0067.jpg, GPS Latitude: 10.895855, GPS Longitude: 106.69395166666666
Image: DJI_0079.jpg, GPS Latitude: 10.895965, GPS Longitude: 106.69385333333334
Image: DJI_0065.jpg, GPS Latitude: 10.895903333333333, GPS Longitude: 106.69530666666667
Image: DJI_0071.jpg, GPS Latitude: 10.8959833333

In [32]:
# Replace 'your_folder_path' with the actual path to the folder containing images.
folder_path = '/content/drive/MyDrive/face/dataset-plastic1/train/images'

# Create a folium map centered at a specific location (you can adjust the latitude and longitude)
map_center = [10.895855, 106.69395833333333]
my_map = folium.Map(location=map_center, )

# Loop through each file in the folder and its subdirectories
for root, _, files in os.walk(folder_path):
    for filename in files:
        image_path = os.path.join(root, filename)
        coordinates = get_gps_coordinates(image_path)

        if coordinates:
            latitude, longitude = coordinates
            popup_text = f"Image: {filename}, GPS Latitude: {latitude}, GPS Longitude: {longitude}"
            folium.Marker(
                location=[latitude, longitude],
                popup=popup_text,
            ).add_to(my_map)

# Display the map
my_map

In [33]:
pip install gmaps


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076090 sha256=f062618371e615c55c4b9d85c0327fd09bc6981bc0e83f68e1fd2840c50a51e9
  Stored in directory: /root/.cache/pip/wheels/b3/c2/dc/48b3ef16c2184dae51a003f17eb5d065bbbf1af3437d9f14e3
Successfully built gmaps


In [42]:
import gmaps
import os
from collections.abc import Iterable

In [44]:
# Replace 'your_folder_path' with the actual path to the folder containing images.
folder_path = '/content/drive/MyDrive/face/dataset-plastic1/train/images'
# Create a map centered at a specific location (you can adjust the latitude and longitude)
map_center = (10.895855, 106.69395833333333)
my_map = gmaps.figure(center=map_center, zoom_level=21)  # Increase the zoom level to 21

# Loop through each file in the folder and its subdirectories
for root, _, files in os.walk(folder_path):
    for filename in files:
        image_path = os.path.join(root, filename)
        coordinates = get_gps_coordinates(image_path)

        if coordinates:
            latitude, longitude = coordinates
            popup_text = f"Image: {filename}, GPS Latitude: {latitude}, GPS Longitude: {longitude}"
            from collections.abc import Iterable
            marker = gmaps.marker_layer([(latitude, longitude)], info_box_content=[popup_text])
            my_map.add_layer(marker)

# Display the map
gmaps.display(my_map)

AttributeError: ignored